# Lesson5 画像からキャプションを生成してみよう

## 目次
- Section 3 テクニック・発展的内容
    - 3.1 Attention
    - 3.2 Beam Search

## Section 3 テクニック・発展的内容

### 3.1 Attention

翻訳モデルでAttentionを使用したのとほぼ同様に、キャプション生成においてもアテンションは使われます。

イメージは次の図の通りです。翻訳モデルではEncoderの各隠れ層$h^e$にたいしてattendしていましたが、キャプション生成ではEncoder (CNN)の出力の$u$に対してattendしていきます。

<img src="./figures/attention-2.png" width="700mm">

翻訳のときと同様に
1. $u$と$h_t$からアテンションスコアの計算
$$ \text{score}(u_s, h_t) = h^T_t W u_s $$
2. $\text{softmax}$操作によりアテンションの重みの計算
$$
    a_t(s) = \frac{\exp(\mathrm{score}(u_s, h_t))}{\sum^S_{s'=1}\exp(\mathrm{score}(u_s, h_t))}
$$
3. 文脈ベクトルの計算
$$
    c_t = \sum^S_{s=1} a_t(s) u_s
$$

の順で行います。

### 3.2 Beam Search Decoding

これまで、テストデータに対して新たな文を生成する際、各ステップで最も確率の高い単語を正解として採用し、次ステップの入力としてつかっていました。

ただ本当にやりたいことは、文全体としての尤度が最も高くなるような文を生成することです。

各ステップで尤度最大の単語を選択したからと言って、文全体での尤度が最大となるとは限りません。

そのため、近視眼的に確率の高い単語を採用していくより、もうすこし大局的に評価していく必要があります。

このような大局的な評価を実現するDecoding手法として、Beam Search Decodingがあります。

Beam Search Decodingは、各ステップでそこまでの対数尤度が高い文を一定数$K$ ($=2\sim10$)個保持しながら選択をおこなうというものです。

（ちなみに、これまでの$K=1$に相当するDecoding手法を特に貪欲Decodingと呼ぶことがあります）

このようにして、より長い系列を見渡して尤度を評価することでより適切な文を生成することが可能になります。

イメージを以下に示します。

<img src="./figures/beam_search_example.png" width="500mm">

ここでは$K=4$で、各ステップでスコアの高いパスを4つ保持しながら出力を行っています。

具体的には,
- t=1でスコアの高い4単語 (thai、police、thailand、three) を採用
- t=2で4単語それぞれに対して4つずつスコアの高い計16単語を取得し、そのなかから累計スコア の高い4パス (thai->police、police->arrest、thailand->arrests、three->suspects)を採用
- t=3で4パスそれぞれに対して4つずつスコアの高い計16単語を取得し、そのなかから累計スコアの高い4パス (thai->police->arrest、police->arrest->three、police->arrest->suspects、police->arrest->suspected)を採用
- ...

のような感じで計算をおこない、最終的に最も累計スコアの高いパス (図青線) を最終的な出力として採用しています。

細かい話にはなりますが、対数尤度で測ると短い系列が有利になりやすいという問題に対処するため、各パスの対数尤度をパス長で割るなどして正規化したスコアで比べる、というTipsも有効です。